In [ ]:
#********************************************************************
# Install packages
#
# NOTE: Uncomment and run each cell once as needed then re-comment
#********************************************************************

In [ ]:
#!pip install arcticdb --user

In [ ]:
#!pip install openbb --user 

In [ ]:
#********************************************************************
# Imports
#********************************************************************
import pandas as pd
from datetime import datetime, timedelta
from arcticdb import Arctic, QueryBuilder
from openbb import obb

In [ ]:
#********************************************************************
# (Optional) Set Jupyter Notebook display
#********************************************************************
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#********************************************************************
# Load OpenBB Personal Access Token (PAT)
#
# NOTE: You need to have previously created an account on OpenBB Hub
#       and generated your Personal Access Token (PAT).
#       Get your PAT here: https://my.openbb.co/app/platform/pat
#********************************************************************
import json

with open('GH_OpenBB_Credentials.json') as f:
    data = json.load(f)
    obb_pat = data['OpenBB_PAT']

In [ ]:
#********************************************************************
# Create OpenBB connection
#********************************************************************
obb.account.login(pat=obb_pat)

In [ ]:
#********************************************************************
# Use OpenBB & YFinance to load data for a stock/index symbol into a Pandas dataframe
#********************************************************************
spx_daily = obb.equity.price.historical("^SPX", start_date="1995-01-01", provider="yfinance").to_df()

display(len(spx_daily))

spx_daily

In [ ]:
#********************************************************************
# Establish connection to ArcticDB 
#
# NOTE: The last segment in the path ("stocks") is the "database instance" name
# NOTE: Use LMDB - this is a local file-based database instance
#********************************************************************
ac = Arctic('lmdb://Databases/stocks?map_size=2GB')

In [ ]:
#********************************************************************
# Establish an ArcticDB library connection
#
# NOTE: Think of the library as a "database" ("history") in the "database instance" ("stocks")
#********************************************************************
lib = ac.get_library('history', create_if_missing=True)

In [ ]:
#********************************************************************
# Save the data to an ArcticDB symbol
#
# NOTE: In ArcticDB, think of a symbol as a "table"
#********************************************************************
table_name = "spx_daily"

# NOTE: Re-running will overwrite all existing data in table_name symbol
write_records = lib.write(table_name, spx_daily)

In [ ]:
#********************************************************************
# Clear out the dataframe to allow demonstration of loading from ArcticDB
#********************************************************************
spx_daily = None

In [ ]:
#********************************************************************
# Read the saved data from ArcticDB
#********************************************************************
read_records = lib.read(table_name) # arcticdb.version_store._store.VersionedItem

display(len(read_records.data))

In [ ]:
#********************************************************************
# Assign dataframe variable to dataframe read in from ArcticDB
#********************************************************************
spx_daily = read_records.data

display(len(spx_daily))

spx_daily

In [ ]:
#********************************************************************
# You can query the data with with the familiarity of Pandas and the efficiency of C++
#
# NOTE: For more information please check out our QueryBuilder docs: 
#       https://docs.arcticdb.io/latest/api/query_builder/
#********************************************************************
q = QueryBuilder()

q = q[(q["close"] > 4500) & (q["close"] < 4550)]

filtered = lib.read(table_name, query_builder=q).data

filtered.head(5)

In [ ]:
#********************************************************************
# You can select out rows and columns efficiently...necessary when the data doesn't fit into ram
#********************************************************************
subframe = lib.read(
    table_name,
    columns=["close", "volume"],
    date_range=(datetime(2023, 1, 4, 0), datetime(2023, 1, 6, 0))
).data

subframe.head(5)